In [ ]:
import pandas as pd
import numpy as np
from ln_utils import *
from transaction_simulator import *

In [ ]:
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt

### WHAT TO DO WITH INF-s???

In [ ]:
experiment_id = "2019-05-30_18:55:02_100000sat_k10000"
snapshots = range(7)
simulation_dir = "/mnt/idms/fberes/data/bitcoin_ln_research/simulations/"

In [ ]:
experiment_files = get_experiment_files(experiment_id, snapshots, simulation_dir)

In [ ]:
all_infos = []
for snap_id in snapshots:
    info, df = aggregate_samples(experiment_files, snap_id)
    all_infos.append(info)
    print(snap_id)

all_infos[0]

### Average correlation (over time) between cost changes and other metric

In [ ]:
methods = ["spearman","wkendall","kendall"]
fig, axis = plt.subplots(len(methods), 1, figsize=(10,15))
for i, m in enumerate(methods):
    mx = corr_mx(all_infos[0].drop("removed_node", axis=1), method=m)
    for j in range(1,len(all_infos)):
        mx = np.add(mx, corr_mx(all_infos[j].drop("removed_node", axis=1), method=m))
    axis[i].set_title(m)
    sns.heatmap(mx / len(all_infos), annot=True, ax=axis[i], vmin=-0.3, vmax=1.0)

# Node analysis with 1ML background

In [ ]:
from scrapy.crawler import CrawlerProcess
from scraping import *

### Average metrics of nodes

In [ ]:
mean_all_info = pd.concat(all_infos, sort=True).groupby("removed_node").mean()
mean_all_info = mean_all_info.sort_values("cost_diff", ascending=False)

def show_df_with_alias(df):
    nodes = list(df.index)
    process = CrawlerProcess()
    process.crawl(LNNodeParser(nodes))
    process.start()
    print()

show_df_with_alias(mean_all_info.head(10))

In [ ]:
mean_all_info.head(10)

In [ ]:
mean_all_info.tail(10)